In [1]:
experiment_name = "baseline.classifier.cnn_4_retinas.sig56_vs_bg"


In [2]:
%load_ext autoreload
%autoreload 2
%env THEANO_FLAGS="device=gpu3"

import sys,os
sys.path.append("..")


env: THEANO_FLAGS="device=gpu3"


In [3]:
#import theano stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


## Preprocess/Load data

In [4]:
from config import from_data
data_path_sig = from_data("sig56/")
data_path_bg_up = from_data("bg_up/")
data_path_bg_down = from_data("bg_down/")
metadata_path = from_data("k0s_event_identification")


generate_metadata = not os.path.exists(metadata_path)
if generate_metadata:
    print "Metadata does not exist. This run might be MUCH longer due to preprocessing the data."

Metadata does not exist. This run might be MUCH longer due to preprocessing the data.


In [ ]:
from lib.retina_compiled import retina_view, retinize_events
import random
get_random_ids = lambda path,k: random.sample(set(map(lambda s: s.split('.')[0],os.listdir(path))), k)



retina_images_path = os.path.join(metadata_path,"retina_images.npy")
answers_path = os.path.join(metadata_path,"sig_or_bg.npy")

if generate_metadata:
    


    param_df = pd.DataFrame.from_csv(from_data("retinas/sig_vs_bg_0.csv"))
    retinas = [retina_view(*params) for params in param_df.values]


    
    sample_names = ["signal","bg_up","bg_down"]

    sample_sizes = [10000,5000,5000]

    y = map(np.repeat, sample_names,sample_sizes)
    y = np.concatenate(y)

    data_paths = [data_path_sig,data_path_bg_up,data_path_bg_down]

    random.seed(7331)
    #event ids : "{runnumber}_{eventnumber}"
    data_runids = map(get_random_ids,data_paths,sample_sizes)

    data_X = [
        retinize_events(ids_i,path_i,retina_views=retinas,
                                 max_hits_block=15000,report_rate=100)
        for ids_i,path_i in zip(data_runids,data_paths)
    ]

    X = np.concatenate(data_X)


    y = np.concatenate(map(np.repeat, sample_names,sample_sizes))
    
    
    os.mkdir(metadata_path)
    np.save(retina_images_path,X)
    np.save(answers_path, y)
    
    generate_metadata = False
else:
    #load metadata
    X = np.load(retina_images_path)
    y = np.load(answers_path)

processing events: 100/10000
processing events: 200/10000
processing events: 300/10000
processing events: 400/10000
processing events: 500/10000
processing events: 600/10000
processing events: 700/10000
processing events: 800/10000
processing events: 900/10000
processing events: 1000/10000
processing events: 1100/10000
processing events: 1200/10000
processing events: 1300/10000
processing events: 1400/10000
processing events: 1500/10000
processing events: 1600/10000
processing events: 1700/10000
processing events: 1800/10000
processing events: 1900/10000
processing events: 2000/10000
processing events: 2100/10000
processing events: 2200/10000
processing events: 2300/10000
processing events: 2400/10000
processing events: 2500/10000
processing events: 2600/10000
processing events: 2700/10000
processing events: 2800/10000
processing events: 2900/10000
processing events: 3000/10000
processing events: 3100/10000
processing events: 3200/10000
processing events: 3300/10000
processing events: 

In [6]:
X = X.reshape([len(X),-1,32,32])

X = X.astype("float32")
y = (y=="signal").astype('int32')

In [7]:
from sklearn.cross_validation import train_test_split
Xtr, Xts, Ytr,Yts = train_test_split(X,y,test_size=0.25,random_state=1337)

print Ytr.shape,Yts.shape

(4500,) (1500,)


# define NN

In [8]:
import lasagne
import theano
import theano.tensor as T

floatX = theano.config.floatX


In [9]:
input_dim = (None,)+X.shape[1:]

retina_dim = (None,1) + X.shape[2:]

retina_images = T.tensor4("input_images","floatX")

any_interesting_decays = T.ivector("event_has_k0s")

In [10]:
concatenated_retinas = retina_images.reshape([retina_images.shape[0],1,-1,retina_images.shape[-1]])

concatenated_dim = (None,1,np.prod(X.shape[1:-1]),X.shape[-1])

In [11]:
from lasagne.layers import InputLayer
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import MaxPool2DLayer as PoolLayer
from lasagne.layers import DenseLayer, DropoutLayer

#nn where each retina is processed with a separate CNN

net = {}

net['input'] = InputLayer(concatenated_dim,input_var=concatenated_retinas)
net['conv1_1'] = ConvLayer(
            net['input'], 256, 5, pad=1, flip_filters=False)
net['pool1'] = PoolLayer(net['conv1_1'], 3)
net['conv2_1'] = ConvLayer(
            net['pool1'], 256, 5, pad=1, flip_filters=False)
net['pool2'] = PoolLayer(net['conv2_1'], 3)
    
net['fc6'] = DenseLayer(net["pool2"], num_units=1024)
#net['fc6_dropout'] = DropoutLayer(net['fc6'], p=0.1)

net['out'] = DenseLayer(net['fc6'], num_units=2, 
                        nonlinearity=lasagne.nonlinearities.softmax)



In [12]:
weights = lasagne.layers.get_all_params(net["out"],trainable=True)
weights

[W, b, W, b, W, b, W, b]

# train updates

In [13]:
train_prediction = lasagne.layers.get_output(net["out"])
train_loss_ce = lasagne.objectives.categorical_crossentropy(train_prediction,any_interesting_decays).mean()
train_accuracy = lasagne.objectives.categorical_accuracy(train_prediction,any_interesting_decays).mean()
updates = lasagne.updates.rmsprop(train_loss_ce,weights,learning_rate=0.001) 
#ik that lr does not matter. I just dont want an explosion.

In [14]:
train_fun = theano.function([retina_images,any_interesting_decays],[train_loss_ce,train_accuracy,train_prediction[:,1]], updates=updates)

# evaluation

In [15]:
prediction = lasagne.layers.get_output(net["out"],deterministic=True)
loss_ce = lasagne.objectives.categorical_crossentropy(prediction,any_interesting_decays).mean()
accuracy = lasagne.objectives.categorical_accuracy(prediction,any_interesting_decays).mean()

In [16]:
eval_fun = theano.function([retina_images,any_interesting_decays], [loss_ce,accuracy,prediction[:,1]])
predict_fun = theano.function([retina_images],prediction)

# main loop
* almost copies the layout of lasagne basic training 

In [17]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False,crop_at=None):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if crop_at == start_idx:
            break
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        


In [18]:
from collections import defaultdict
from sklearn.metrics import roc_auc_score
metrics = defaultdict(dict)

import time
num_epochs = 50
batch_size = 50

# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    y_pred_batches = []
    y_ref_batches = []
    for batch in iterate_minibatches(Xtr, Ytr, batch_size, shuffle=True):
        batch_ce,batch_acc,pred_batch= train_fun(*batch)
        train_err +=batch_ce
        train_acc +=batch_acc
        train_batches += 1
        y_pred_batches.append(pred_batch)
        y_ref_batches.append(batch[1])
        
    y_pred = np.concatenate(y_pred_batches)
    y_ref = np.concatenate(y_ref_batches)
    train_auc = roc_auc_score(y_ref,y_pred)

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    y_pred_batches = []
    y_ref_batches = []
    for batch in iterate_minibatches(Xts, Yts, batch_size, shuffle=False):
        err, acc,pred_batch = eval_fun(*batch)
        val_err += err
        val_acc += acc
        val_batches += 1
        y_pred_batches.append(pred_batch)
        y_ref_batches.append(batch[1])

    y_pred = np.concatenate(y_pred_batches)
    y_ref = np.concatenate(y_ref_batches)
    val_auc = roc_auc_score(y_ref,y_pred)

    metrics["acc_train"][epoch] = train_acc/train_batches
    metrics["auc_train"][epoch] = train_auc
    metrics["acc_test"][epoch] = val_acc/val_batches
    metrics["auc_test"][epoch] = val_auc
    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  training accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches))
    print("  training ROC AUC:\t\t{:.3f} %".format(
        train_auc))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.3f} %".format(
        val_acc / val_batches))
    print("  validation ROC AUC:\t\t{:.3f} %".format(
        val_auc))



Epoch 1 of 100 took 10.665s
  training loss:		89.153786
  training accuracy:		0.54 %
  training ROC AUC:		0.548 %
  validation loss:		0.694064
  validation accuracy:		0.497 %
  validation ROC AUC:		0.519 %
Epoch 2 of 100 took 10.718s
  training loss:		3.622323
  training accuracy:		0.52 %
  training ROC AUC:		0.528 %
  validation loss:		0.671854
  validation accuracy:		0.564 %
  validation ROC AUC:		0.738 %
Epoch 3 of 100 took 10.748s
  training loss:		0.957583
  training accuracy:		0.57 %
  training ROC AUC:		0.581 %
  validation loss:		0.625092
  validation accuracy:		0.607 %
  validation ROC AUC:		0.794 %
Epoch 4 of 100 took 10.758s
  training loss:		0.839664
  training accuracy:		0.62 %
  training ROC AUC:		0.651 %
  validation loss:		0.493290
  validation accuracy:		0.782 %
  validation ROC AUC:		0.861 %
Epoch 5 of 100 took 10.775s
  training loss:		0.740147
  training accuracy:		0.67 %
  training ROC AUC:		0.726 %
  validation loss:		0.453494
  validation accuracy:		0.793 %
  val

KeyboardInterrupt: 

In [ ]:
for metric_name in metrics:
    
    plt.plot(*zip(*sorted(metrics[metric_name].items())),label = metric_name)
plt.legend()

# Evaluation

In [ ]:
val_err = 0
val_acc = 0
val_batches = 0
y_pred_batches = []
y_ref_batches = []

for batch in iterate_minibatches(Xts, Yts, batch_size, shuffle=False):
    err, acc,pred_batch = eval_fun(*batch)
    val_err += err
    val_acc += acc
    val_batches += 1
    y_pred_batches.append(pred_batch)
    y_ref_batches.append(batch[1])

y_pred = np.concatenate(y_pred_batches)
y_ref = np.concatenate(y_ref_batches)
val_auc = roc_auc_score(y_ref,y_pred)

print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
print("  validation accuracy:\t\t{:.3f} %".format(
    val_acc*100 / val_batches))
print("  validation ROC AUC:\t\t{:.3f} %".format(
    val_auc*100))
